<a href="https://colab.research.google.com/github/Kasel04/GenAI/blob/main/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kasel *Carty*

In [69]:
from google.colab import files
files.upload()  # Choose kaggle.json


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kaselcarty","key":"e78635db7293bce4f4eebb143fd85958"}'}

In [62]:

!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [72]:
!kaggle datasets download -d jhoward/lsun_bedroom --force

Dataset URL: https://www.kaggle.com/datasets/jhoward/lsun_bedroom
License(s): ODbL-1.0
100% 8.88G/8.89G [02:04<00:00, 203MB/s]
100% 8.89G/8.89G [02:04<00:00, 76.8MB/s]


In [80]:
!ls /content


kaggle.json  lsun_bedroom  lsun_bedroom_subset	lsun_bedroom.zip  sample_data


In [75]:
!unzip lsun_bedroom.zip -d lsun_bedroom


Streaming output truncated to the last 5000 lines.
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd5fedec227994c1816b44f2ca9f3e82c6d7443.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd6160588fd3dc9e06b4288690d1d9ee0b541c6.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd6c84b1a3d3db0749222146c7fa4b5311f8d9f.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd6f96b894e3dedbcb629e7ed9f8f5b32b70581.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd713f5f784867f8e0d6eb45af0b24e2b98dcdd.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd7217f9d2fba8cea938a35c928741e361911fd.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd742d13cde6954f0a18564a91c8b901be140e2.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd7a210da182506c1cde48d7585bef758056f5d.jpg  
  inflating: lsun_bedroom/sample/data0/lsun/bedroom/f/b/d/fbd7d7c87c4c622c539cab278cf0516fa666ec3f.jpg  
  in

In [83]:
!ls /content/lsun_bedroom


data0  sample


In [84]:
!ls /content/lsun_bedroom/*/


/content/lsun_bedroom/data0/:
lsun

/content/lsun_bedroom/sample/:
data0


In [86]:
import random
import shutil
import os

# Set the paths
FULL_DATASET_PATH = '/content/lsun_bedroom/data0/lsun'  # Corrected to point to the 'lsun' folder
SUBSET_PATH = '/content/lsun_bedroom_subset'  # Path to save the subset

# Define the number of images you want in your subset
NUM_IMAGES_IN_SUBSET = 1000  # You can adjust this value

# Create the subset directory if it doesn't exist
os.makedirs(SUBSET_PATH, exist_ok=True)

# List all images in the full dataset directory (adjust if necessary)
all_images = []
for root, dirs, files in os.walk(FULL_DATASET_PATH):  # Traverse all subdirectories
    for file in files:
        if file.endswith(('.jpg', '.jpeg', '.png')):  # Filter for image files
            all_images.append(os.path.join(root, file))

# Ensure that NUM_IMAGES_IN_SUBSET does not exceed the number of available images
if NUM_IMAGES_IN_SUBSET > len(all_images):
    print(f"Warning: Requested subset size ({NUM_IMAGES_IN_SUBSET}) exceeds the available images ({len(all_images)}). Using {len(all_images)} images instead.")
    NUM_IMAGES_IN_SUBSET = len(all_images)

# Randomly sample images to create a subset
subset_images = random.sample(all_images, NUM_IMAGES_IN_SUBSET)

# Copy selected images to the subset directory
for img in subset_images:
    shutil.copy(img, SUBSET_PATH)

print(f"Created a subset with {NUM_IMAGES_IN_SUBSET} images at {SUBSET_PATH}.")




Created a subset with 1000 images at /content/lsun_bedroom_subset.


In [4]:
# Check and create subdirectory if necessary
if not os.path.exists(DATASET_PATH):
    os.makedirs(DATASET_PATH)

subdirectory = os.path.join(DATASET_PATH, 'data0')
if not os.path.exists(subdirectory):
    os.makedirs(subdirectory)

# Move images into 'data0' subdirectory if they aren’t already
for img in os.listdir(DATASET_PATH):
    img_path = os.path.join(DATASET_PATH, img)
    if os.path.isfile(img_path):
        shutil.move(img_path, subdirectory)


In [5]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Specify the directory path for the images
DATASET_PATH = '/content/lsun_bedroom_subset'  # Updated to use the subset


# Define preprocessing parameters
IMG_SIZE = 64
BATCH_SIZE = 256

# Use ImageDataGenerator for loading and normalizing images
datagen = ImageDataGenerator(rescale=1.0/127.5, preprocessing_function=lambda x: x - 1.0)
dataset = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode=None
)


Found 0 images belonging to 1 classes.


In [6]:
import tensorflow as tf

def make_generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(100,)))  # Define the input shape here
    model.add(tf.keras.layers.Dense(8*8*256, use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Reshape((8, 8, 256)))
    model.add(tf.keras.layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(64, 64, 3)))  # Define the input shape here
    model.add(tf.keras.layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))
    return model


In [7]:
# Define the loss functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Set up optimizers
generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)


In [8]:
generator = make_generator_model()
discriminator = make_discriminator_model()


In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import os


In [10]:
# Define a single training step
@tf.function

def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Define the training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)


In [11]:
import matplotlib.pyplot as plt

def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))

    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i + 1)
        plt.imshow((predictions[i] + 1) / 2)
        plt.axis('off')

    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.show()


In [ ]:
# Set parameters
epochs = 50  # number of epochs
noise_dim = 100
seed = tf.random.normal([16, noise_dim])  # Seed for generating images

# Train the GAN
train(dataset, epochs)

# Generate and save images after training
generate_and_save_images(generator, epochs, seed)



Discussion

1. There’s a balance between making images look good and making them different. If the model focuses too much on quality, the images might start looking the same. We want it to show different bedroom styles while having it still looking realistic.
2. Yes, experimenting with different hyperparameters, like learning rate and batch size, can help the model perform better. Small adjustments may improve image quality, training stability, and diversity in the generated images.
3. This type of generative model can be used in interior design/real estate (for example room layouts), video game design (for realistic backgrounds), or VR environments where unique, realistic images are needed quickly.
4. (Extra Credit) The Inception Score measures how realistic images are and whether they capture a range of classes, while Inception Distance compares generated images to real ones, measuring both quality and diversity.